In [ ]:
# sudo apt install fuse-zip opus-tools

In [ ]:
# pip3 install librosa soundfile numpy pandas tqdm

In [110]:
import re
import os
import pydub
import librosa
import soundfile
import subprocess
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm

In [128]:
!opusenc SA1.wav 9.ogg

Error parsing input file: SA1.wav


In [ ]:
!wget -O timit.zip https://ndownloader.figshare.com/files/10256148

In [112]:
!mkdir -p timit && fuse-zip -r timit.zip timit

In [113]:
TIMIT_PATH = 'timit/data/lisa/data/timit/raw/TIMIT'

In [114]:
path = f'{TIMIT_PATH}/DOC/SPKRINFO.TXT'
lines = open(path, 'r').read().splitlines()
header = lines[37].replace(' ', '  ')[1:]
lines = [header] + lines[39:]
pad = lambda s: s if len(s) > 52 else s + '  -'
stc = lambda s: re.sub('[ ]{2,}', ',', s)
cts = lambda s: s.replace(',', ';')
lines = [stc(cts(pad(line))) for line in lines]
open('spkrinfo.csv', 'w').write('\n'.join(lines));

In [134]:
wav_paths = glob(f'{TIMIT_PATH}/*/*/*/*.WAV')

In [135]:
# getting rid of stupid text headers in wav files
for wav_path in tqdm(wav_paths):
    audio, sr = soundfile.read(wav_path)
    new_path = wav_path.replace(TIMIT_PATH, 'wav')
    os.makedirs('/'.join(new_path.split('/')[:-1]), exist_ok=True)
    soundfile.write(new_path, audio, sr)

100%|███████████████████████████████████████████████████████████████████████████████████| 6300/6300 [00:32<00:00, 192.55it/s]


In [136]:
wav_paths = glob(f'wav/*/*/*/*.WAV')

In [139]:
for wav_path in tqdm(wav_paths):
    ogg_path = wav_path.replace('wav', 'ogg').replace('WAV','ogg')
    os.makedirs('/'.join(ogg_path.split('/')[:-1]), exist_ok=True)
    Popen(['opusenc', '--quiet', wav_path, ogg_path]).wait()
    # audio, sr = soundfile.read(wav_path)
    # soundfile.write(ogg_path, audio, sr)

100%|████████████████████████████████████████████████████████████████████████████████████| 6300/6300 [06:41<00:00, 15.68it/s]


In [ ]:
# sudo umount timit && rm timit.zip && rm -rf timit && rm -rf wav

In [140]:
df = pd.read_csv('spkrinfo.csv')
df = df.drop(columns=['Ht','Race','Edu','Comments'])
df.columns = ['id','sex','dr','use','rec_date','birth_date']

def get_year(date):
    s = date.str.split('/').str[-1]
    return s.replace('??', 'nan').astype(float)

def get_duration(wavfile):
    return librosa.get_duration(filename=wavfile)

def get_audio_files(path):
    return sorted(os.listdir(f'ogg/{path}'))

yr = get_year(df['rec_date'])
yb = get_year(df['birth_date'])
df['age'] = (yr - yb).fillna(50)
df['use'] = df['use'].map({'TRN':'TRAIN','TST':'TEST'})
df['dr'] = 'DR' + df['dr'].astype(str)
df['path'] = df['use'] + '/' + df['dr'] + '/' + df['sex'] + df['id']
df['audio'] = df['path'].apply(get_audio_files)
df = df.explode('audio')

df['audio'] = df['path'] + '/' + df['audio']
df['duration'] = ('ogg/' + df['audio']).apply(get_duration)
df = df.drop(columns=['dr','use','path'])
df.to_csv('spkrinfo.csv', index=False)

In [141]:
!mv ogg timit && mv spkrinfo.csv timit/ && zip -r -q timit.zip timit

In [ ]:
!gdown --id 1VTEsfOMCGBbhyhWtQitsRzN9dW5UUyIy